In [1]:
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE TemplateHaskell #-}

import AdventOfCode
import Control.Lens

import qualified Data.Attoparsec.ByteString.Char8 as A
import qualified Data.Map.Strict as M
import qualified Data.Vector as V

input <- dayLines 18

In [2]:
type Reg = Char
type ValueReg = Either Int Reg

parseReg = A.satisfy A.isAlpha_ascii
parseValue = A.signed A.decimal

parseValueReg :: A.Parser ValueReg
parseValueReg = A.eitherP parseValue parseReg

In [3]:
data Instr 
    = Snd ValueReg 
    | Set Reg ValueReg
    | Add Reg ValueReg
    | Mul Reg ValueReg
    | Mod Reg ValueReg
    | Rcv Reg
    | Jgz ValueReg ValueReg
    deriving (Eq, Show)

parseInstr = A.choice
    [ Snd <$> ("snd" *> pVR)
    , Set <$> ("set" *> pR)  <*> pVR
    , Add <$> ("add" *> pR)  <*> pVR
    , Mul <$> ("mul" *> pR)  <*> pVR
    , Mod <$> ("mod" *> pR)  <*> pVR
    , Rcv <$> ("rcv" *> pR)
    , Jgz <$> ("jgz" *> pVR) <*> pVR
    ]
    where
        pVR = A.space *> parseValueReg
        pR  = A.space *> parseReg

registerMap = M.fromList $ zip ['a'..'p'] (repeat 0)
instrVec    = V.fromList $ map (parsed parseInstr) input

In [4]:
data MachineState = MachineState
    { _registers :: M.Map Char Int
    , _counter   :: Int
    } deriving (Eq, Show)

makeLenses ''MachineState

getValueReg :: M.Map Char Int -> ValueReg -> Int
getValueReg registers = either id (\r -> registers ^. at r . non 0)

incr = counter +~ 1

executeInstr instr ms = case instr of
    Snd     vr -> ms & incr
    Set reg vr -> ms & incr & registers %~ (\rs -> rs & at reg ?~       (getValueReg rs vr))
    Add reg vr -> ms & incr & registers %~ (\rs -> rs & ix reg %~     (+(getValueReg rs vr)))
    Mul reg vr -> ms & incr & registers %~ (\rs -> rs & ix reg %~     (*(getValueReg rs vr)))
    Mod reg vr -> ms & incr & registers %~ (\rs -> rs & ix reg %~ (`mod`(getValueReg rs vr)))
    Jgz vr vr' -> if getValueReg (ms ^. registers) vr > 0
        then ms & counter %~ (+ (getValueReg (ms ^. registers) vr'))
        else ms & incr
    other -> error $ "don't know how to execute " ++ show other

In [5]:
loop :: V.Vector Instr -> MachineState -> Int -> Int
loop instrs ms lastFreq = case instrs ^? ix (ms ^. counter) of
    Nothing       -> lastFreq
    Just (Snd vr) -> loop instrs (executeInstr (Snd vr) ms) (getValueReg (ms ^. registers) vr)
    Just (Rcv r)  -> if (ms ^. registers . at r . non 0) /= 0 then lastFreq else loop instrs (ms & incr) lastFreq
    Just curr     -> loop instrs (executeInstr curr ms) lastFreq

In [6]:
loop instrVec (MachineState registerMap 0) 0

3423

In [7]:
data LockedState = Running | Waiting deriving (Eq, Show)
data ConcMachineState = ConcMachineState
    { _ms          :: MachineState
    , _mailbox     :: [Int]
    , _lockedState :: LockedState
    }

makeLenses ''ConcMachineState

send msg cms = cms & mailbox %~ (|> msg)

executeInstr' instr cms = case instr of
    Rcv reg -> if (cms ^. mailbox) == [] 
        then cms & lockedState .~ Waiting
        else cms 
                & ms %~ (\m -> m & incr & registers %~ (at reg ?~ (cms ^. mailbox . to head))) 
                & mailbox %~ tail 
                & lockedState .~ Running
    _ -> cms & ms %~ (executeInstr instr)

loop' :: V.Vector Instr -> ConcMachineState -> ConcMachineState -> Int -> Int
loop' instrs cms0 cms1 count
    | cms0 ^. lockedState == Waiting && cms1 ^. lockedState == Waiting             = count
    | Nothing `elem` map (\cms -> instrs ^? ix (cms ^. ms . counter)) [cms0, cms1] = count
    | Just (Snd vr) <- instrs ^? ix (cms0 ^. ms . counter) = let
        value = getValueReg (cms0 ^. ms . registers) vr
        cms1' = send value cms1
        cms0' = executeInstr' (Snd vr) cms0
        in loop' instrs cms0' cms1' count
    | Just (Snd vr) <- instrs ^? ix (cms1 ^. ms . counter) = let
        value = getValueReg (cms1 ^. ms . registers) vr
        cms0' = send value cms0
        cms1' = executeInstr' (Snd vr) cms1
        in loop' instrs cms0' cms1' (count + 1)
    | Just instr0 <- instrs ^? ix (cms0 ^. ms . counter)
    , Just instr1 <- instrs ^? ix (cms1 ^. ms . counter) = let
        cms0' = executeInstr' instr0 cms0
        cms1' = executeInstr' instr1 cms1
        in loop' instrs cms0' cms1' count

initialMs0 = ConcMachineState (MachineState  registerMap                0) [] Running
initialMs1 = ConcMachineState (MachineState (registerMap & at 'p' ?~ 1) 0) [] Running

In [8]:
loop' instrVec initialMs0 initialMs1 0

7493